### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Sport climbing at the Summer Olympics

Competitions in the sport of climbing, governed by the International Federation of Sport Climbing (IFSC), have been held at two editions of the Summer Olympic Games. First selected as one of the discretionary sports at the 2020 and 2024 games, sport climbing will be inducted as one of the mandatory sports at the 2028 games. 

Athletes compete in the disciplines of BOULDERING, LEAD Climbing, and SPEED climbing. All three disciplines were contested as a single event in the 2020 programme : so 2 climbing events were contested (men's combined and women's combined). This decision caused widespread criticism in the climbing community. While speed climbing was spun off into its own event in the 2024 programme, and combined is now BOULDERING and LEAD Climbing

In [2]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_Staging'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [3]:
### url
Wiki_Climbing_Summer_Olympics = "https://en.wikipedia.org/wiki/Sport_climbing_at_the_Summer_Olympics"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Climbing_Summer_Olympics = pd.read_html(Wiki_Climbing_Summer_Olympics)
type(Dataset_Wiki_Climbing_Summer_Olympics), len(Dataset_Wiki_Climbing_Summer_Olympics)

(list, 18)

##### on récupère 1 list de 18 élts

In [27]:
type(Dataset_Wiki_Climbing_Summer_Olympics[0])

pandas.core.frame.DataFrame

##### > Chaque élt est un DataFrame Pandas
##### ? Quel est le contenu chacun des 18 dataframes ? Lesquelles nous interessent ?

In [28]:
Dataset_Wiki_Climbing_Summer_Olympics[0]

,Sport climbing at the Summer Olympics,Sport climbing at the Summer Olympics.1
0,NaN,NaN
1,IOC Code,CLB
2,Governing body,IFSC
3,Events,4 (men: 2; women: 2)
4,Summer Olympics,Summer Olympics
5,1896 1900 1904 1908 1912 1920 1924 1928 1932 1...,1896 1900 1904 1908 1912 1920 1924 1928 1932 1...
6,Medalists,Medalists


######## [0] is dataset of summarized data about Sport Climbing at Summer Olympics dataset

In [30]:
Dataset_Wiki_Climbing_Summer_Olympics[1]

,Games,Events,Best nations
0,2020 Summer Olympics,2,Spain (1) Slovenia (1)
1,2024 Summer Olympics,4,Poland (1)


######## [1] is the Nb of Events at 2020 and 2024 Summer Olympics and Best Nations

In [32]:
Dataset_Wiki_Climbing_Summer_Olympics[2]

,0,1,2,3
0,Current program,Current program,Current program,Current program
1,Event,2020,2024,Years
2,"Men's combined (s, b, l)",X,NaN,1
3,"Men's combined (b, l)",NaN,X,1
4,Men's speed,NaN,X,1
5,"Women's combined (s, b, l)",X,NaN,1
6,"Men's combined (b, l)",NaN,X,1
7,Women's speed,NaN,X,1
8,NaN,NaN,NaN,NaN
9,Events,2,4,NaN


[2] is about Program of Climbing events at 2020 and 2024 Summer Olympics > NOT to Discard.

In [34]:
Dataset_Wiki_Climbing_Summer_Olympics[3]

,Nation,2020,2024,Years
0,Australia,2,2,2.0
1,Austria,2,2,2.0
2,Belgium,–,1,1.0
3,Canada,2,–,1.0
4,China,2,7,2.0
5,Czech Republic,1,1,2.0
6,France,4,7,2.0
7,Germany,2,3,2.0
8,Great Britain,1,4,2.0
9,Indonesia,–,4,1.0


[3] is the Participating Nations at 2020 and 2024 Summer Olympics

In [6]:
Dataset_Wiki_Climbing_Summer_Olympics[3].dtypes

Nation     object
2020       object
2024       object
Years     float64
dtype: object

In [4]:
Wiki_IFSC_Climbing_Summer_Olympics_Participating_Nations_Df3 = Dataset_Wiki_Climbing_Summer_Olympics[3]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_Participating_Nations_Df3
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Nation VARCHAR(100),
                                        2020 VARCHAR(100),
                                        2024 VARCHAR(100),
                                        Years FLOAT(64),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_Participating_Nations_Df3.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_Participating_Nations_Df3',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

err: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near '2020'. (102) (SQLExecDirectW)")


In [36]:
Dataset_Wiki_Climbing_Summer_Olympics[4]

,Event,Round,Climber,Nation,Time,Games,Date,Record
0,Men's combined (speed),Qualification,Bassa Mawem,France,5.45,2020 Tokyo,3 August 2021,OR[11]
1,Women's combined (speed),Qualification,Aleksandra Mirosław,Poland,6.97,2020 Tokyo,4 August 2021,OR[12]
2,Women's combined (speed),Final,Aleksandra Mirosław,Poland,6.84,2020 Tokyo,6 August 2021,WR[13]
3,Women's speed,Qualification,Aleksandra Mirosław,Poland,6.06,2024 Paris,5 August 2024,WR[14]
4,Men's speed,Bronze medal match,Sam Watson,United States,4.74,2024 Paris,8 August 2024,WR


[4] is the Records history at 2020 and 2024 Summer Olympics > NOT to Discard.

In [7]:
Dataset_Wiki_Climbing_Summer_Olympics[5]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,Slovenia (SLO),2,0,0,2
1,2,Poland (POL),1,0,1,2
2,3,Great Britain (GBR),1,0,0,1
3,3,Indonesia (INA),1,0,0,1
4,3,Spain (ESP),1,0,0,1
5,6,Japan (JPN),0,2,1,3
6,6,United States (USA),0,2,1,3
7,8,China (CHN),0,2,0,2
8,9,Austria (AUT),0,0,3,3
9,Totals (9 entries),Totals (9 entries),6,6,6,18


[5] is the Nations Medal table at 2024 Summer Olympics > NOT to Discard.

In [10]:
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[0,1]

'Slovenia\xa0(SLO)'

In [12]:
# Delete \xa0 and (3 digit Code) 
# and Translate Country Names in French to Match with other Tables de Data (géographiques, Démographiques, ...)
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[0,1] = 'Slovénie'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[1,1] = 'Pologne'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[2,1] = 'Royaume-Uni'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[3,1] = 'Indonésie'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[4,1] = 'Espagne'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[5,1] = 'Japon'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[6,1] = 'États-Unis'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[7,1] = 'Chine'
Dataset_Wiki_Climbing_Summer_Olympics[5].iloc[8,1] = 'Autriche'


In [8]:
Dataset_Wiki_Climbing_Summer_Olympics[5].dtypes

Rank      object
Nation    object
Gold       int64
Silver     int64
Bronze     int64
Total      int64
dtype: object

In [13]:
Dataset_Wiki_Climbing_Summer_Olympics[5]

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,Slovénie,2,0,0,2
1,2,Pologne,1,0,1,2
2,3,Royaume-Uni,1,0,0,1
3,3,Indonésie,1,0,0,1
4,3,Espagne,1,0,0,1
5,6,Japon,0,2,1,3
6,6,États-Unis,0,2,1,3
7,8,Chine,0,2,0,2
8,9,Autriche,0,0,3,3
9,Totals (9 entries),Totals (9 entries),6,6,6,18


In [14]:
Wiki_IFSC_Climbing_Summer_Olympics_2024_Countries_Total_Ranking_Df5 = Dataset_Wiki_Climbing_Summer_Olympics[5]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_2024_Countries_Total_Ranking_Df5
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rank VARCHAR(50),
                                        Nation VARCHAR(50),
                                        Gold INT,
                                        Silver INT,
                                        Bronze INT,
                                        Total INT                                                                  
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_2024_Countries_Total_Ranking_Df5.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_2024_Countries_Total_Ranking_Df5',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [22]:
Dataset_Wiki_Climbing_Summer_Olympics[6]

,Games,Gold,Silver,Bronze
0,2020 Tokyo details,Alberto Ginés López Spain,Nathaniel Coleman United States,Jakob Schubert Austria
1,2024 Paris details,Toby Roberts Great Britain,Sorato Anraku Japan,Jakob Schubert Austria


[6] is about Men Medal winner full names and Nations in  COMBINED's discipline at 2020 and 2024 Summer Olympics

In [23]:
# Replace 'Games' by 'Year' to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[6]. columns = ['Year','Gold','Silver','Bronze']
Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[0,0] = 2020
Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[1,0] = 2024
Dataset_Wiki_Climbing_Summer_Olympics[6]

,Year,Gold,Silver,Bronze
0,2020,Alberto Ginés López Spain,Nathaniel Coleman United States,Jakob Schubert Austria
1,2024,Toby Roberts Great Britain,Sorato Anraku Japan,Jakob Schubert Austria


In [24]:
# Delete Countries to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[0,1] = 'Alberto Ginés López'

Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[0,2] = 'Nathaniel Coleman'

Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[0,3] = 'Jakob Schubert'

Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[1,1] = 'Toby Roberts'

Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[1,2] = 'Sorato Anraku'

Dataset_Wiki_Climbing_Summer_Olympics[6].iloc[1,3] = 'Jakob Schubert'

Dataset_Wiki_Climbing_Summer_Olympics[6]

,Year,Gold,Silver,Bronze
0,2020,Alberto Ginés López,Nathaniel Coleman,Jakob Schubert
1,2024,Toby Roberts,Sorato Anraku,Jakob Schubert


In [12]:
Dataset_Wiki_Climbing_Summer_Olympics[6].dtypes

Year      object
Gold      object
Silver    object
Bronze    object
dtype: object

In [25]:
Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Combined_Df6 = Dataset_Wiki_Climbing_Summer_Olympics[6]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Combined_Df6
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Combined_Df6.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Combined_Df6',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [42]:
Dataset_Wiki_Climbing_Summer_Olympics[7]

,Games,Gold,Silver,Bronze
0,2024 Paris details,Veddriq Leonardo Indonesia,Wu Peng China,Sam Watson United States


[7] is about Men Medal winner full names and Nations in Speed's discipline at 2024 Summer Olympics > NOT to Discard.

In [26]:
# Replace 'Games' by 'Year' to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[7]. columns = ['Year','Gold','Silver','Bronze']
Dataset_Wiki_Climbing_Summer_Olympics[7].iloc[0,0] = 2024
Dataset_Wiki_Climbing_Summer_Olympics[7]

,Year,Gold,Silver,Bronze
0,2024,Veddriq Leonardo Indonesia,Wu Peng China,Sam Watson United States


In [27]:
# Delete Countries to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[7].iloc[0,1] = 'Veddriq Leonardo'

Dataset_Wiki_Climbing_Summer_Olympics[7].iloc[0,2] = 'Wu Peng'

Dataset_Wiki_Climbing_Summer_Olympics[7].iloc[0,3] = 'Sam Watson'

Dataset_Wiki_Climbing_Summer_Olympics[7]

,Year,Gold,Silver,Bronze
0,2024,Veddriq Leonardo,Wu Peng,Sam Watson


In [30]:
Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Speed_Df7 = Dataset_Wiki_Climbing_Summer_Olympics[7]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Speed_Df7
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Speed_Df7.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_Men_Medal_Speed_Df7',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [44]:
Dataset_Wiki_Climbing_Summer_Olympics[8]

,Games,Gold,Silver,Bronze
0,2020 Tokyo details,Janja Garnbret Slovenia,Miho Nonaka Japan,Akiyo Noguchi Japan
1,2024 Paris details,Janja Garnbret Slovenia,Brooke Raboutou United States,Jessica Pilz Austria


[8] is about Women Medal winner full names and Nations in COMBINED's discipline at 2020 and 2024 Summer Olympics > NOT to Discard.

In [31]:
# Replace 'Games' by 'Year' to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[8]. columns = ['Year','Gold','Silver','Bronze']
Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[0,0] = 2020
Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[1,0] = 2024
Dataset_Wiki_Climbing_Summer_Olympics[8]

,Year,Gold,Silver,Bronze
0,2020,Janja Garnbret Slovenia,Miho Nonaka Japan,Akiyo Noguchi Japan
1,2024,Janja Garnbret Slovenia,Brooke Raboutou United States,Jessica Pilz Austria


In [32]:
# Delete Countries to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[0,1] = 'Janja Garnbret'

Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[0,2] = 'Miho Nonaka'

Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[0,3] = 'Akiyo Noguchi'

Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[1,1] = 'Janja Garnbret'

Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[1,2] = 'Brooke Raboutou '

Dataset_Wiki_Climbing_Summer_Olympics[8].iloc[1,3] = 'Jessica Pilz'

Dataset_Wiki_Climbing_Summer_Olympics[8]

,Year,Gold,Silver,Bronze
0,2020,Janja Garnbret,Miho Nonaka,Akiyo Noguchi
1,2024,Janja Garnbret,Brooke Raboutou,Jessica Pilz


In [35]:
Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Combined_Df8 = Dataset_Wiki_Climbing_Summer_Olympics[8]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Combined_Df8
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Combined_Df8.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Combined_Df8',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [46]:
Dataset_Wiki_Climbing_Summer_Olympics[9]

,Games,Gold,Silver,Bronze
0,2024 Paris details,Aleksandra Mirosław Poland,Deng Lijuan China,Aleksandra Kałucka Poland


[9] is about Women Medal winner full names and Nations in Speed's discipline at 2024 Summer Olympics > NOT to Discard.

In [33]:
# Replace 'Games' by 'Year' to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[9]. columns = ['Year','Gold','Silver','Bronze']
Dataset_Wiki_Climbing_Summer_Olympics[9].iloc[0,0] = 2024
Dataset_Wiki_Climbing_Summer_Olympics[9]

,Year,Gold,Silver,Bronze
0,2024,Aleksandra Mirosław Poland,Deng Lijuan China,Aleksandra Kałucka Poland


In [34]:
# Delete Countries to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics[9].iloc[0,1] = 'Aleksandra Mirosław'

Dataset_Wiki_Climbing_Summer_Olympics[9].iloc[0,2] = 'Deng Lijuan'

Dataset_Wiki_Climbing_Summer_Olympics[9].iloc[0,3] = 'Aleksandra Kałucka'

Dataset_Wiki_Climbing_Summer_Olympics[9]

,Year,Gold,Silver,Bronze
0,2024,Aleksandra Mirosław,Deng Lijuan,Aleksandra Kałucka


In [36]:
Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Speed_Df9 = Dataset_Wiki_Climbing_Summer_Olympics[9]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Speed_Df9
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Year VARCHAR(100),
                                        Gold VARCHAR(100),
                                        Silver VARCHAR(100),
                                        Bronze VARCHAR(100),                                                                   
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Speed_Df9.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_Women_Medal_Speed_Df9',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [48]:
Dataset_Wiki_Climbing_Summer_Olympics[10]

,vteSport climbing at the Summer Olympics,vteSport climbing at the Summer Olympics.1,vteSport climbing at the Summer Olympics.2,vteSport climbing at the Summer Olympics.3
0,NaN,1896–2016 2020 2024,1896–2016 2020 2024,NaN
1,List of medalists List of venues,List of medalists List of venues,List of medalists List of venues,List of medalists List of venues


[10] > to Discard

In [49]:
Dataset_Wiki_Climbing_Summer_Olympics[11]

,vteSports at the Olympic Games,vteSports at the Olympic Games.1
0,Summer sports,Aquatics Artistic swimming Diving Swimming Wat...
1,Winter sports,Biathlon Bobsleigh Bobsleigh Skeleton Curling ...
2,2026 addition,Ski mountaineering
3,2028 additions,Baseball / Softball Cricket Flag football Lacr...
4,Past sports,Basque pelota Boxing Breaking Croquet Jeu de p...
5,Demonstration sports,American football Australian football Bandy Bu...
6,Unofficial sports,Aeronautics Alpinism Angling Art competitions ...
7,Paralympic sports Olympic Games portal,Paralympic sports Olympic Games portal


In [50]:
Dataset_Wiki_Climbing_Summer_Olympics[12]

,vteClimbing competitions,vteClimbing competitions.1
0,Disciplines,Lead climbing (also called sport climbing) Bou...
1,Rankings,Ranking of career IFSC victories by climber
2,Rock,WorldIFSC IFSC Climbing World Cup IFSC Climbin...
3,World,IFSC IFSC Climbing World Cup IFSC Climbing Wor...
4,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
5,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...
6,Continental,IFSC African Championships African Youth Champ...
7,IFSC,African Championships African Youth Championsh...
8,Multi-sport,Asian Games European Games
9,National,American Championships Austrian Championships ...


[12]  is NOT about Climbing > to discard !!

In [51]:
Dataset_Wiki_Climbing_Summer_Olympics[13]

,0,1
0,World,IFSC IFSC Climbing World Cup IFSC Climbing Wor...
1,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
2,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...
3,Continental,IFSC African Championships African Youth Champ...
4,IFSC,African Championships African Youth Championsh...
5,Multi-sport,Asian Games European Games
6,National,American Championships Austrian Championships ...
7,Individual,Sportroccia Rock Master Melloblocco Nordic Cha...


[13]  is NOT about Climbing > to discard !!

In [52]:
Dataset_Wiki_Climbing_Summer_Olympics[14]

,0,1
0,IFSC,IFSC Climbing World Cup IFSC Climbing World Ch...
1,Multi-sport,Summer Olympics Summer Youth Olympics World Ga...


[14]  is NOT about Climbing > to discard !!

In [53]:
Dataset_Wiki_Climbing_Summer_Olympics[15]

,0,1
0,IFSC,African Championships African Youth Championsh...
1,Multi-sport,Asian Games European Games


[15] is NOT about Climbing > to discard !!

In [54]:
Dataset_Wiki_Climbing_Summer_Olympics[16]

,0,1
0,World,UIAA Ice Climbing World Cup UIAA Ice Climbing ...


[16] is NOT about Climbing > to discard !!